In [1]:
import numpy as np 
import pandas as pd 
import tensorflow as tf 
import keras
from keras.layers import Dense, LSTM, SimpleRNN, GRU, Input
from keras.models import Model, Sequential  

df_train = pd.read_csv("sales_train_validation.csv")
#df_sell = pd.read_csv("./storage/Kaggle_M5_Luke/sell_prices.csv")
#df_calendar = pd.read_csv("./storage/Kaggle_M5_Luke/calendar.csv")
#df_ss = pd.read_csv("./storage/Kaggle_M5_Luke/sample_submission_m5.csv")


Using TensorFlow backend.


In [2]:
days = range(1,1913+1) 
time_series_cols = [] 
for i in days: 
    time_series_cols.append('d_'+str(i)) 
time_series_cols[:10]

['d_1', 'd_2', 'd_3', 'd_4', 'd_5', 'd_6', 'd_7', 'd_8', 'd_9', 'd_10']

In [3]:
time_series_data = df_train[time_series_cols] 
time_series_data

,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,0,0,0,0,0,0,0,0,0,0,...,1,3,0,1,1,1,3,0,1,1
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,2,1,2,1,1,1,0,1,1,1
3,0,0,0,0,0,0,0,0,0,0,...,1,0,5,4,1,0,1,3,7,2
4,0,0,0,0,0,0,0,0,0,0,...,2,1,1,0,1,1,2,2,2,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,0,0,2,2,0,3,1,4,1,0,...,2,0,0,0,0,0,1,0,0,1
30486,0,0,0,0,0,5,0,1,1,3,...,0,0,0,0,0,0,0,0,1,0
30487,0,6,0,2,2,4,1,8,5,2,...,2,1,0,2,0,1,0,0,1,0
30488,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,1,0,3,1,3


In [4]:
time_series_data = np.asarray(time_series_data).astype('float32')
print(time_series_data.shape)

(30490, 1913)


In [5]:
mean = [] # list of mean values 
sd = [] # list of standard deviations 

# normalize row by row 
for i in range(len(time_series_data)):  
    mean.append(np.mean(time_series_data[i])) 
    sd.append(np.std(time_series_data[i])) 
    time_series_data[i] = (time_series_data[i]-np.mean(time_series_data[i]))/np.std(time_series_data[i]) 

In [6]:
time_series_data 

array([[-0.45782614, -0.45782614, -0.45782614, ..., -0.45782614,
         1.0018762 ,  1.0018762 ],
       [-0.4527011 , -0.4527011 , -0.4527011 , ..., -0.4527011 ,
        -0.4527011 , -0.4527011 ],
       [-0.30949783, -0.30949783, -0.30949783, ...,  1.7462986 ,
         1.7462986 ,  1.7462986 ],
       ...,
       [-0.64613056,  3.6858127 , -0.64613056, ..., -0.64613056,
         0.07585997, -0.64613056],
       [-0.38167205, -0.38167205, -0.38167205, ...,  2.7385786 ,
         0.65841144,  2.7385786 ],
       [-0.46050826, -0.46050826, -0.46050826, ..., -0.46050826,
        -0.46050826, -0.46050826]], dtype=float32)

In [7]:
def build_model(time_steps,features): 
    model = Sequential() 
    model.add(LSTM(128,input_shape=(time_steps,features),activation='relu')) 
    model.add(Dense(30490))   
    return model 

In [10]:
# let's now code for the general case: for all the 28 days 
def split_xy2(dataset, time_steps):
    x,y = list(), list() 
    for i in range(0,dataset.shape[1]-time_steps): 
        x.append(dataset[:,i:i+time_steps]) 
        y.append(dataset[:,i+time_steps]) 
    return np.array(x), np.array(y) 

predictions = [] 

for i in range(1,29): 
    print("day {}".format(i)) 
    dataset = time_series_data[:,::i]
    x_train, y_train = split_xy2(dataset,7) 
    x_pred = dataset[:,dataset.shape[1]-7:]  
    x_train = x_train.reshape(x_train.shape[0],7,30490) 
    x_pred = x_pred.reshape(1,7,30490)  
    model = build_model(7,30490) 
    model.compile(loss='mse',optimizer='adam',metrics=['mse'])
    model.fit(x_train,y_train,epochs=50) 
    pred = model.predict(x_pred)   
    for i in range(len(pred[0])):  
        pred[0][i] *= sd[i] 
        pred[0][i] += mean[i]
    print(pred[0]) # for checking purposes 
    predictions.append(pred) 
    


day 1
Epoch 1/50
1906/1906 [==============================] - 3s 2ms/step - loss: 0.9182 - mse: 0.9182
Epoch 2/50
1906/1906 [==============================] - 2s 1ms/step - loss: 0.8910 - mse: 0.8910
Epoch 3/50
1906/1906 [==============================] - 3s 1ms/step - loss: 0.8813 - mse: 0.8813
Epoch 4/50
1906/1906 [==============================] - 2s 1ms/step - loss: 0.8697 - mse: 0.8697
Epoch 5/50
1906/1906 [==============================] - 3s 1ms/step - loss: 0.8586 - mse: 0.8586
Epoch 6/50
1906/1906 [==============================] - 3s 1ms/step - loss: 0.8565 - mse: 0.8565
Epoch 7/50
1906/1906 [==============================] - 2s 1ms/step - loss: 0.8583 - mse: 0.8583
Epoch 8/50
1906/1906 [==============================] - 2s 1ms/step - loss: 0.8549 - mse: 0.8549
Epoch 9/50
1906/1906 [==============================] - 2s 1ms/step - loss: 0.8579 - mse: 0.8579
Epoch 10/50
1906/1906 [==============================] - 2s 1ms/step - loss: 0.8543 - mse: 0.8543
Epoch 11/50
1906/1906 [

In [11]:
predictions = np.asarray(predictions) 
print(predictions.shape)

(28, 1, 30490)


In [12]:
predictions

array([[[0.73757184, 0.18346912, 0.43241987, ..., 0.7898297 ,
         0.9353243 , 1.2187902 ]],

       [[0.48186386, 0.21813872, 0.2439315 , ..., 0.90061206,
         0.579974  , 1.058547  ]],

       [[0.43469226, 0.2799135 , 0.20685206, ..., 0.90222824,
         0.5540229 , 0.8228447 ]],

       ...,

       [[1.1744735 , 0.3109989 , 0.49543738, ..., 0.42043096,
         0.7500646 , 1.936336  ]],

       [[1.8959637 , 0.54068244, 0.06123784, ..., 0.17355895,
         1.1023782 , 1.843596  ]],

       [[0.35993373, 0.26341325, 0.17426337, ..., 0.71189874,
         0.4835998 , 1.0276895 ]]], dtype=float32)

In [13]:
# flatten each of predictions[i] 
pred_flattened = [] 
for i in range(28): 
    pred_flattened.append(predictions[i].flatten()) 

In [14]:
# checking if shape is flattened 
pred_flattened[0]

array([0.73757184, 0.18346912, 0.43241987, ..., 0.7898297 , 0.9353243 ,
       1.2187902 ], dtype=float32)

In [15]:
cols = [] 
for i in range(1,29): 
    cols.append("F"+str(i)) 
df = pd.DataFrame(pred_flattened)   
df = df.T # transpose 
df.columns = cols # rename column 
df 

,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,0.737572,0.481864,0.434692,0.802136,0.869925,0.582867,0.727921,0.368073,0.393875,0.345531,...,0.619406,0.585528,0.318169,-0.139548,0.501964,1.481895,0.416332,1.174474,1.895964,0.359934
1,0.183469,0.218139,0.279914,0.295112,0.182056,0.251819,0.428215,0.313518,0.353979,0.459230,...,0.092470,0.146009,0.279601,0.474372,0.254075,-0.022252,0.067836,0.310999,0.540682,0.263413
2,0.432420,0.243932,0.206852,0.189426,0.450445,0.326285,0.672797,0.211131,0.336836,0.142396,...,0.093637,0.553713,0.192089,0.320372,0.179043,0.231184,0.114118,0.495437,0.061238,0.174263
3,1.948367,1.545887,1.896145,1.570567,1.761485,1.442754,2.170097,1.853847,2.279135,1.013444,...,1.614750,1.929142,1.772731,1.652490,1.899802,6.515775,1.228752,1.178680,1.423558,1.651676
4,0.924299,0.918755,0.837950,1.020844,1.002761,0.825549,1.239159,1.550868,1.199142,1.149803,...,-0.039681,0.723402,0.956479,0.373167,0.850490,1.355114,0.575831,0.844676,0.572235,0.918150
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,0.446569,0.369155,0.358607,0.443402,0.310433,0.340505,0.387421,0.544053,0.393282,0.622377,...,0.304000,0.328769,0.531000,0.547800,0.348016,0.568338,0.107259,0.015597,0.078739,0.538480
30486,0.170742,0.268169,0.268282,0.225023,0.240117,0.188974,0.304815,0.217580,0.221026,0.362606,...,0.416636,0.179288,0.357401,0.589349,0.191320,0.089876,0.587120,-0.058471,0.171393,0.317075
30487,0.789830,0.900612,0.902228,0.869707,0.721938,0.981413,0.941885,0.300979,0.805111,0.887936,...,1.292930,0.693207,0.938234,0.786549,1.126284,0.533411,1.614911,0.420431,0.173559,0.711899
30488,0.935324,0.579974,0.554023,0.716445,0.804688,0.828524,0.417447,0.299251,0.539292,0.375720,...,1.066873,1.030536,0.327435,0.781600,0.499938,2.106688,0.207188,0.750065,1.102378,0.483600


In [16]:
validation_ids = df_train['id'].values
evaluation_ids = [i.replace('validation', 'evaluation') for i in validation_ids]
ids = np.concatenate([validation_ids, evaluation_ids])       
ids

array(['HOBBIES_1_001_CA_1_validation', 'HOBBIES_1_002_CA_1_validation',
       'HOBBIES_1_003_CA_1_validation', ...,
       'FOODS_3_825_WI_3_evaluation', 'FOODS_3_826_WI_3_evaluation',
       'FOODS_3_827_WI_3_evaluation'], dtype=object)

In [17]:
sub_df = pd.DataFrame(ids, columns=['id']) 
df = pd.concat([df]*2).reset_index(drop=True) 
sub_df = pd.concat([sub_df,df],axis=1) 
sub_df.to_csv('LSTM_128_kth_standardized.csv',index=False) 

In [18]:
sub_df 

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_validation,0.737572,0.481864,0.434692,0.802136,0.869925,0.582867,0.727921,0.368073,0.393875,...,0.619406,0.585528,0.318169,-0.139548,0.501964,1.481895,0.416332,1.174474,1.895964,0.359934
1,HOBBIES_1_002_CA_1_validation,0.183469,0.218139,0.279914,0.295112,0.182056,0.251819,0.428215,0.313518,0.353979,...,0.092470,0.146009,0.279601,0.474372,0.254075,-0.022252,0.067836,0.310999,0.540682,0.263413
2,HOBBIES_1_003_CA_1_validation,0.432420,0.243932,0.206852,0.189426,0.450445,0.326285,0.672797,0.211131,0.336836,...,0.093637,0.553713,0.192089,0.320372,0.179043,0.231184,0.114118,0.495437,0.061238,0.174263
3,HOBBIES_1_004_CA_1_validation,1.948367,1.545887,1.896145,1.570567,1.761485,1.442754,2.170097,1.853847,2.279135,...,1.614750,1.929142,1.772731,1.652490,1.899802,6.515775,1.228752,1.178680,1.423558,1.651676
4,HOBBIES_1_005_CA_1_validation,0.924299,0.918755,0.837950,1.020844,1.002761,0.825549,1.239159,1.550868,1.199142,...,-0.039681,0.723402,0.956479,0.373167,0.850490,1.355114,0.575831,0.844676,0.572235,0.918150
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60975,FOODS_3_823_WI_3_evaluation,0.446569,0.369155,0.358607,0.443402,0.310433,0.340505,0.387421,0.544053,0.393282,...,0.304000,0.328769,0.531000,0.547800,0.348016,0.568338,0.107259,0.015597,0.078739,0.538480
60976,FOODS_3_824_WI_3_evaluation,0.170742,0.268169,0.268282,0.225023,0.240117,0.188974,0.304815,0.217580,0.221026,...,0.416636,0.179288,0.357401,0.589349,0.191320,0.089876,0.587120,-0.058471,0.171393,0.317075
60977,FOODS_3_825_WI_3_evaluation,0.789830,0.900612,0.902228,0.869707,0.721938,0.981413,0.941885,0.300979,0.805111,...,1.292930,0.693207,0.938234,0.786549,1.126284,0.533411,1.614911,0.420431,0.173559,0.711899
60978,FOODS_3_826_WI_3_evaluation,0.935324,0.579974,0.554023,0.716445,0.804688,0.828524,0.417447,0.299251,0.539292,...,1.066873,1.030536,0.327435,0.781600,0.499938,2.106688,0.207188,0.750065,1.102378,0.483600


In [16]:
#model.compile(loss='mse',optimizer='adam',metrics=['mse'])
#model_checkpoint = keras.callbacks.ModelCheckpoint('./storage/Kaggle_M5_Luke/model_base2/model{epoch:02d}.h5',period=50)
#model.fit(x_train,y_train,epochs=1000,callbacks=[model_checkpoint],validation_data=(x_val,y_val))  
